# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import model_selection, datasets, linear_model, metrics
import numpy as np
import pandas as pd

### Генерация датасета

In [2]:
iris = datasets.load_iris()

In [3]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                    test_size=0.3, random_state=0)

### Задание модели

In [4]:
classifier = linear_model.SGDClassifier(random_state=0, tol=1e-3)

### Генерация сетки

In [5]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [6]:
parameters_grid = {
    'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty': ['l1', 'l2'],
    'max_iter': np.arange(5, 10),
    'alpha': np.linspace(0.0001, 0.001, num=5),
}

In [7]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

### Подбор параметров и оценка качества

#### Grid search

In [8]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring='accuracy', cv=cv)

In [9]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 3.75 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [13]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [15]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


#### Randomized grid search

In [19]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring='accuracy', cv=cv, n_iter=20, 
                                                        random_state=0)

In [21]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 459 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [22]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}
